<a href="https://colab.research.google.com/github/rakeshpuppala2590/Market-Anomaly-Detection/blob/main/MAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [6]:
data = pd.read_csv('/content/FinancialMarketData.csv')

In [7]:
data.head()

,Y,Data,XAU BGNL,ECSURPUS,BDIY,CRY,DXY,JPY,GBP,Cl1,...,LP01TREU,EMUSTRUU,LF94TRUU,MXUS,MXEU,MXJP,MXBR,MXRU,MXIN,MXCN
0,0,1/11/2000,283.25,0.077,1388.0,157.26,100.56,105.86,1.646,25.77,...,116.464,230.527,123.762,1416.12,127.75,990.59,856.76,224.33,217.34,34.30
1,0,1/18/2000,287.65,0.043,1405.0,165.01,101.86,105.47,1.638,28.85,...,117.267,231.377,123.762,1428.79,129.50,993.98,925.22,234.37,227.08,32.74
2,0,1/25/2000,287.15,0.135,1368.0,167.24,102.41,106.04,1.650,28.28,...,117.995,232.390,123.762,1385.93,126.48,974.83,886.93,216.82,233.00,32.46
3,0,2/1/2000,282.75,0.191,1311.0,166.85,104.92,107.85,1.611,28.22,...,120.510,231.942,122.328,1385.31,129.19,1007.12,842.60,201.89,237.48,31.29
4,1,2/8/2000,298.40,0.312,1277.0,165.43,104.22,109.30,1.611,28.02,...,118.791,237.812,122.328,1411.95,134.67,1034.58,945.15,218.00,258.02,31.32


In [9]:
print("\nDataset Info:")
print(data.info())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 44 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Y          1111 non-null   int64  
 1   Data       1111 non-null   object 
 2   XAU BGNL   1111 non-null   float64
 3   ECSURPUS   1111 non-null   float64
 4   BDIY       1111 non-null   float64
 5   CRY        1111 non-null   float64
 6   DXY        1111 non-null   float64
 7   JPY        1111 non-null   float64
 8   GBP        1111 non-null   float64
 9   Cl1        1111 non-null   float64
 10  VIX        1111 non-null   float64
 11  USGG30YR   1111 non-null   float64
 12  GT10       1111 non-null   float64
 13  USGG2YR    1111 non-null   float64
 14  USGG3M     1111 non-null   float64
 15  US0001M    1111 non-null   float64
 16  GTDEM30Y   1111 non-null   float64
 17  GTDEM10Y   1111 non-null   float64
 18  GTDEM2Y    1111 non-null   float64
 19  EONIA      1111 non-null   float6

In [14]:
data = data.dropna()

In [15]:
features = data.drop(['Y'], axis=1)
features = features.select_dtypes(include=['float64'])
target = data['Y']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [19]:
y_pred = model.predict(X_test)


In [20]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[170   7]
 [ 13  33]]


In [21]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.96      0.94       177
           1       0.82      0.72      0.77        46

    accuracy                           0.91       223
   macro avg       0.88      0.84      0.86       223
weighted avg       0.91      0.91      0.91       223



In [22]:
print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

Accuracy Score:
0.9103139013452914


In [23]:
import joblib
joblib.dump(model, 'anomaly_detection_model.pkl')

def propose_strategy(predictions):
    for idx, prediction in enumerate(predictions):
        if prediction == 1:
            print(f"Day {idx + 1}: Crash predicted! Suggest risk mitigation strategies.")
        else:
            print(f"Day {idx + 1}: No crash predicted. Continue with standard investment strategies.")

propose_strategy(y_pred)

Day 1: No crash predicted. Continue with standard investment strategies.
Day 2: Crash predicted! Suggest risk mitigation strategies.
Day 3: Crash predicted! Suggest risk mitigation strategies.
Day 4: No crash predicted. Continue with standard investment strategies.
Day 5: No crash predicted. Continue with standard investment strategies.
Day 6: Crash predicted! Suggest risk mitigation strategies.
Day 7: No crash predicted. Continue with standard investment strategies.
Day 8: Crash predicted! Suggest risk mitigation strategies.
Day 9: No crash predicted. Continue with standard investment strategies.
Day 10: No crash predicted. Continue with standard investment strategies.
Day 11: No crash predicted. Continue with standard investment strategies.
Day 12: No crash predicted. Continue with standard investment strategies.
Day 13: No crash predicted. Continue with standard investment strategies.
Day 14: Crash predicted! Suggest risk mitigation strategies.
Day 15: No crash predicted. Continue w